In [1]:
import pandas as pd 
import numpy as np 
from scipy.sparse import bsr_matrix

In [2]:
anime = pd.read_csv('../data/datasets_571_1094_anime.csv')
rating = pd.read_csv('../data/rating_1.csv')

In [3]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [4]:
rating.head()

,user_id,anime_id,rating
0,1,8074,10
1,1,11617,10
2,1,11757,10
3,1,15451,10
4,2,11771,10


### Playing around with scipy dense matrix data structures 

In [5]:
# row = np.array([0, 0, 1, 2, 2, 2])
# col = np.array([0, 2, 2, 0, 2, 2])
# data = np.array([1, 2, 3 ,4, 5, 6])
# mat = bsr_matrix((data, (row, col)), shape=(3, 3))
# mat.toarray()

### Creating my own sparse matrix

In [6]:
anime_id = anime['anime_id'].unique()
columns = np.sort(anime_id)

user_id = rating['user_id'].unique()
index = np.sort(user_id)

In [7]:
sparse_df = pd.DataFrame(columns=columns)
sparse_df

,1,5,6,7,8,15,16,17,18,19,...,34498,34501,34502,34503,34506,34514,34519,34522,34525,34527


In [8]:
grp_rating = rating.groupby(by='user_id')

In [9]:
# Create dict to match column name (anime_id) to index for row_of_zeros
col_len = len(columns)
indx = list(range(0,col_len))
row_of_zeros = np.zeros(col_len, dtype=int)
users = rating['user_id'].unique()

In [10]:
index_dict = dict()
for c, i in zip(columns, indx):
    index_dict.update({c:i})

In [11]:
# instead of appending to the df create a matrix and use that as the data to create the dataframe
# Then use the data from to create a scipy sparse matrix
# data = np.array([], dtype=int)
data = []
for user in users:
    try:
        row_to_append = np.copy(row_of_zeros)
        group = grp_rating.get_group(user)
        put_index = [index_dict.get(x) for x in group['anime_id']]
        put_values = [x for x in group['rating']]
        np.put(row_to_append, put_index, put_values)
        data.append([row_to_append])
    except TypeError as e:
        print(f'Error {e} \nUser {user}')

Error int() argument must be a string, a bytes-like object or a number, not 'NoneType' 
User 34240
Error int() argument must be a string, a bytes-like object or a number, not 'NoneType' 
User 37442


In [12]:
len(data)

69598

In [13]:
len(rating['user_id'].unique())

69600

In [14]:
# import pickle
# with open('../data/sparse_matrix.pckl', 'wb') as f:
#     pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

In [15]:
sm_df = pd.DataFrame(data=data, columns=columns)
sm_df.head()

ValueError: 12294 columns passed, passed data had 1 columns

-------------------
## Testing Area
-----------------

In [16]:
zeros = np.zeros(10)
print(f'Before put {zeros}| length {len(zeros)}')
np.put(zeros, 1, 5)
print(f'After put {zeros}| length {len(zeros)}')

Before put [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]| length 10
After put [0. 5. 0. 0. 0. 0. 0. 0. 0. 0.]| length 10


In [17]:
group_1 = grp_rating.get_group(1)
group_1

,user_id,anime_id,rating
0,1,8074,10
1,1,11617,10
2,1,11757,10
3,1,15451,10


In [18]:
test_df = pd.DataFrame(columns=list(range(0,10)))
test_series = pd.Series(data=[1,1,1,1,1,1,1,1,1,1])
# test_df = test_df.append(test_series, ignore_index=True)
test_df = test_df.append([[1,1,1,1,1,1,1,1,1,1]], ignore_index=True)
test_df = test_df.append([[1,1,1,1,1,1,1,1,1,1]], ignore_index=True)
test_df = test_df.append([[1,1,1,1,1,1,1,1,1,1]], ignore_index=True)
test_df

,0,1,2,3,4,5,6,7,8,9
0,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1


In [19]:
ap_test = np.zeros(10, dtype=int)
np.put(ap_test, 1, 9)
ap_test

array([0, 9, 0, 0, 0, 0, 0, 0, 0, 0])

In [20]:
test_data = np.array([], dtype=int)
test_data = np.append(test_data, [1])

In [21]:
test_data = np.append(test_data, [1])
test_data

array([1, 1])